In [1]:
from reservoir_rls import *
from lorenz63 import *
from sklearn import preprocessing
from scipy.stats import sem
import cma
from scipy.stats import kstest, norm, shapiro
from multiprocessing import Pool
from min_func import *
from tqdm import tqdm


from jupyterthemes import jtplot
jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)

In [2]:
step = 0.05
start_time = 125
sync_length = 200
num_tests = 500
train_length = 800
pred_length = 400
base_data_length = int(start_time/step)
data_length = int(base_data_length * step/(0.01))+pred_length
r_t = lambda x: r_t_cosine(x)
dxdt = lambda x,t,r_t: dxdt_lorenz(x,t,r_t)

num_tests = 100
with tqdm(total = num_tests) as pbar:
    for seed in range(num_tests):
        data = getLorenzData(data_length, r_t, dxdt, sample_tau = step, seed = seed)
        if seed == 0:
            optim_data = data[-sync_length - train_length - pred_length:]
        else:
            optim_data = np.vstack((optim_data, data[-sync_length - train_length - pred_length:]))
        pbar.update(1)
print(optim_data.shape)

# scaler = preprocessing.StandardScaler().fit(data)
# scaled_data = scaler.transform(data)
scaled_data = np.copy(optim_data)
scaled_data = np.ascontiguousarray(scaled_data)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:12<00:00,  1.37it/s]

(140000, 3)


In [3]:
num_nodes = 210
res_seed = 1
base_res = reservoir(3,num_nodes,input_weight = 1, spectral_radius = 1, seed = res_seed)
mask = ['input_weight','regularization']
x0 = np.array([5, 5])
min_func = lambda x: vt_min_function_norm(scaled_data, x, mask, base_res.Win, base_res.A, \
     num_tests = num_tests,  num_nodes = num_nodes, pred_length = pred_length, train_length = train_length,
     separated = True)
sigma = 2

In [4]:
opts = cma.CMAOptions()
opts.set('popsize',6*x0.size)
opts.set('bounds', [0,10])
opts.set('seed', 5)
opts.set('verb_filenameprefix','cmaes_norm_cos_starttime%d_res%d\\' % (start_time,res_seed))
results = cma.fmin(min_func, x0, sigma, options = opts)

(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 2 (seed=5, Tue Dec 15 14:37:41 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     12 -3.200000000000000e+01 1.0e+00 1.81e+00  1e+00  2e+00 0:41.4
    2     24 -3.300000000000000e+01 1.7e+00 2.48e+00  1e+00  4e+00 1:21.1
    3     36 -2.950000000000000e+01 2.6e+00 2.26e+00  9e-01  3e+00 2:00.7
    4     48 -2.500000000000000e+01 3.9e+00 1.78e+00  6e-01  2e+00 2:39.3
    5     60 -3.150000000000000e+01 4.2e+00 1.67e+00  4e-01  3e+00 3:09.9
    6     72 -3.400000000000000e+01 7.4e+00 1.81e+00  3e-01  3e+00 3:39.8
    7     84 -3.350000000000000e+01 1.2e+01 1.49e+00  2e-01  2e+00 4:09.6
    8     96 -3.350000000000000e+01 1.2e+01 1.22e+00  1e-01  2e+00 4:38.7
    9    108 -3.350000000000000e+01 1.4e+01 9.55e-01  7e-02  1e+00 5:08.1
   10    120 -3.200000000000000e+01 1.8e+01 7.86e-01  5e-02  9e-01 5:37.7
   11    132 -3.400000000000000e+01 1.8e+01 6.99e-01  4e-02  7e-01 6:07.7
   12    144 -3.400000000000

In [ ]:
input_weight = 5.066848935533839
regs = np.linspace(2,8,25)
forgets = np.linspace(0,5,25)
func_vals = np.zeros((regs.size, forgets.size))

with tqdm(total = regs.size * forgets.size) as pbar:
    for i in range(regs.size):
        for j in range(forgets.size):
            func_vals[i,j] = min_func(np.array([input_weight, regs[i], forgets[j]]))
            np.savetxt('rls_func_vals_weight%f.csv'%input_weight, func_vals, delimiter = ',')
            pbar.update(1)

In [ ]:
plt.pcolor(regs, forgets, -func_vals.T)
plt.colorbar()
plt.show()

In [ ]:
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms
def confidence_ellipse(mean_x, mean_y, cov, ax, n_std=1.0, facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of *x* and *y*.

    Parameters
    ----------
    x, y : array-like, shape (n, )
        Input data.

    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.

    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.

    **kwargs
        Forwarded to `~matplotlib.patches.Ellipse`

    Returns
    -------
    matplotlib.patches.Ellipse
    """
    
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensionl dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0), width=ell_radius_x * 2, height=ell_radius_y * 2,
                      facecolor=facecolor, **kwargs)

    # Calculating the stdandard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    # mean_x = np.mean(x)

    # calculating the stdandard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    # mean_y = np.mean(y)

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    print(ellipse)
    return ax.add_patch(ellipse)

In [ ]:
from matplotlib.animation import FuncAnimation
import pandas as pd
step = 0.05
test_weight = 5.066848935533839

covmats = np.loadtxt('cmaes_norm_cos_starttime125_res1/covmats.txt', delimiter = ',')
mean_data = np.loadtxt('cmaes_norm_cos_starttime125_res1/phenomean.txt', delimiter = ',')[2:]
num_frames = 92
# func_vals = np.loadtxt('rls_func_vals_weight%f.csv'%test_weight, delimiter = ',')*step
mean_func = pd.read_csv('cmaes_rls_const_weight_starttime125_res1/fit.dat', sep = ' ', header = None, skiprows = [0]).to_numpy()[:,5]*step
plt.plot(np.max(np.abs(covmats), axis = 1))
plt.xlim(0,num_frames)
plt.show()

weights = np.linspace(2,8,25)
regs = np.linspace(0,5,25)
W, R = np.meshgrid(weights, regs)

fig = plt.figure()
ax = fig.add_subplot(1,1,1)

def init_base(ax,W,R,func_vals):
    ax.clear()
    # plt.pcolor(W, R, -func_vals.T)
    # plt.colorbar()
    ax.set_xlim(2,8)
    ax.set_ylim(0,5)
    xticks = np.linspace(2,8,7)
    yticks = np.linspace(0,5,6)
    xlabels = ['%.0f'%i for i in -xticks-3.0]
    ylabels = ['%.1f'%i for i in -0.5*yticks - 1.0]
    plt.xticks(xticks, xlabels)
    plt.yticks(yticks, ylabels)
    plt.xlabel('log$_{10}$(Regularization)')
    plt.ylabel('log$_{10}$(1 - Forgetting)')

def animate(i,ax,mean_data,covmats,W, R, func_vals, test_weight):
    ax.clear()
    # plt.pcolor(W, R, -func_vals.T)
    confidence_ellipse(mean_data[i,1],mean_data[i,2],covmats[i,:].reshape(2,2),ax,edgecolor = 'red', linewidth = 2)
    ax.scatter(mean_data[i,0],mean_data[i,1],c = 'red')
    ax.set_xlim(2,8)
    ax.set_ylim(0,5)
    xticks = np.linspace(2,8,7)
    yticks = np.linspace(0,5,6)
    xlabels = ['%.0f'%i for i in -xticks-3.0]
    ylabels = ['%.1f'%i for i in -0.5*yticks - 1.0]
    plt.xticks(xticks, xlabels)
    plt.yticks(yticks, ylabels)
    plt.xlabel('log$_{10}$(Regularization)')
    plt.ylabel('log$_{10}$(1 - Forgetting)')
    plt.title('Color Input Weight: %.3f, Mean Input Weight: %.3f' % ((test_weight-5)*0.2, (test_weight-5)*0.2))
    
update = lambda i: animate(i,ax,mean_data,covmats,W,R,func_vals,test_weight)
init = lambda : init_base(ax,W,R,func_vals)

ani = FuncAnimation(fig, update, frames = num_frames, init_func = init, blit = False)

ani.save('test_rls_cweight.gif')
fig.show()

fig2 = plt.figure()

ax2 = fig2.add_subplot(1,1,1)
line, = ax2.plot([],[],lw = 2, color = 'red')

def init2(ax2, line, num_frames):
    ax2.set_xlim(0,num_frames)
    ax2.set_ylim(0,2.25)
    ax2.set_xlabel('CMA-ES Generation')
    ax2.set_ylabel('Median Valid Time')
    ax2.grid()
    line.set_data([],[])
    return line,

def animate2(i, line, mean_func):
    line.set_data(np.arange(i+1),np.append(np.array([0]),-mean_func[:i]))
    return line,
    

init_fun = lambda : init2(ax2, line, num_frames)    
update2 = lambda i: animate2(i,line, mean_func)

ani2 = FuncAnimation(fig2, update2, frames = num_frames, init_func = init_fun, blit = True)

ani2.save('test2_rls_cweight.gif')



In [ ]:
res2 = reservoir(3,210,forget = 1, input_weight = 0.25*0.05735187596920699, regularization = 10**(-9.310114193664688), seed = 3)
num_tests = 5000
pred_length = 500
train_length = 400
sync_length = 200
valid_times_norm = cross_validation_performance_resync(scaled_data,res2,num_tests,sync_length, \
   train_length, pred_length,train_method = 'Normal', progress = True)
print(np.median(valid_times_norm))

In [ ]:
res1 = reservoir(3,200,forget = 0.995, input_weight = 0.01, LM_regularization = 1e-10, delta = 1e6)
res2 = reservoir(3,200,forget = 1, input_weight = 0.01, regularization = 1e-7)

In [ ]:
data_length = 100000
step = 0.05
r_t = lambda x: r_t_const(x)
dxdt = lambda x,t,r_t: dxdt_lorenz(x,t,r_t)
data = getLorenzData(data_length, r_t, dxdt, sample_tau = step)

sync_length = 200
num_tests = 500
train_length = 800
pred_length = 500

# scaler = preprocessing.StandardScaler().fit(data)
# scaled_data = scaler.transform(data)
scaled_data = np.copy(data)
scaled_data = np.ascontiguousarray(scaled_data)

In [ ]:
valid_times_RLS = cross_validation_performance(scaled_data,res1,num_tests,sync_length, \
   train_length, pred_length,train_method = 'RLS')
print(np.median(valid_times_RLS))
valid_times_norm = cross_validation_performance(scaled_data,res2,num_tests,sync_length, \
   train_length, pred_length,train_method = 'Normal')
print(np.median(valid_times_norm))

In [ ]:
res1 = reservoir(3,300,forget = 0.985, input_weight = 0.01, regularization = 1e-8, \
     t_regularization = 5e-5, delta = 1e6, t_weighted = True)

In [ ]:
# valid_times_norm_array = np.loadtxt('Lorenz63Data/valid_times_norm_tweight_forget%f_reg%e_treg%e.csv' \
#     %(res1.forget, res1.regularization, res1.t_regularization), delimiter = ',')

In [ ]:
step = 0.05
base_data_length = 25000
data_length = int(base_data_length * step/(0.01))
r_t = lambda x: r_t_cosine(x)
dxdt = lambda x,t,r_t: dxdt_lorenz(x,t,r_t)
valid_times_norm_array = np.array([])

sync_length = 1000
num_tests = 300
train_length = 800
pred_length = 200
completed_seeds = valid_times_norm_array.shape[0]

# scaler = preprocessing.StandardScaler().fit(data)
# scaled_data = scaler.transform(data)
from tqdm import tqdm
with tqdm(total = num_tests) as pbar:
    for seed in range(num_tests):
        data = getLorenzData(data_length, r_t, dxdt, sample_tau = step, seed = seed)
        scaled_data = np.ascontiguousarray(data)
        valid_times_norm = cross_validation_performance_versust(scaled_data,res1,sync_length, \
               train_length, pred_length,pred_gap_length = 100, train_method = 'Normal', progress = False)
        if valid_times_norm_array.size == 0:
            valid_times_norm_array = np.copy(valid_times_norm)
        else:
            valid_times_norm_array = np.vstack((valid_times_norm_array,valid_times_norm))
        np.savetxt('Lorenz63Data/valid_times_norm_tweight_forget%f_reg%e_treg%e.csv' %(res1.forget, res1.regularization, res1.t_regularization), \
            valid_times_norm_array, delimiter = ',')
        pbar.update(1)

In [ ]:
np.savetxt('Lorenz63Data/valid_times_norm_tweight_forget%f_reg%e_treg%e.csv' %(res1.forget, res1.regularization, res1.t_regularization), \
    valid_times_norm_array, delimiter = ',')

In [ ]:
plt.plot(np.mean(valid_times_norm_array*0.05,axis = 0))
plt.ylim(0,5)
plt.show()

In [ ]:
res1 = reservoir(3,300,forget = 0.985, input_weight = 0.01, LM_regularization = 1e-10, \
     t_regularization = 0, delta = 1e6, t_weighted = False, RLS_reg_type = 'LM')
step = 0.05
base_data_length = 5000
data_length = int(base_data_length * step/(0.01))
r_t = lambda x: r_t_cosine(x)
dxdt = lambda x,t,r_t: dxdt_lorenz(x,t,r_t)
valid_times_norm_array = np.array([])

sync_length = 500
num_tests = 100
train_length = 2000
pred_length = 400

In [ ]:
data = getLorenzData(data_length, r_t, dxdt, sample_tau = step, seed = 1)
new_data = np.ascontiguousarray(data)

In [ ]:
res1.trainWoutRLS(scaled_data[2800:3800],200)
res1.valid_time(scaled_data[3800:4200],plot = True, error_bound = 3.2)

In [ ]:
valid_times_norm = cross_validation_performance_separated(scaled_data,res1,sync_length, \
               train_length, pred_length, train_method = 'RLS', progress = False)
print(np.median(valid_times_norm))

In [ ]:
# scaler = preprocessing.StandardScaler().fit(data)
# scaled_data = scaler.transform(data)
from tqdm import tqdm
with tqdm(total = num_tests) as pbar:
    for seed in range(num_tests):
        data = getLorenzData(data_length, r_t, dxdt, sample_tau = step, seed = seed)
        scaled_data = np.ascontiguousarray(data)
        valid_times_norm = cross_validation_performance_versust(scaled_data,res1,sync_length, \
               train_length, pred_length,pred_gap_length = 100, train_method = 'RLS', progress = False)
        if valid_times_norm_array.size == 0:
            valid_times_norm_array = np.copy(valid_times_norm)
        else:
            valid_times_norm_array = np.vstack((valid_times_norm_array,valid_times_norm))
        np.savetxt('Lorenz63Data/valid_times_rls_tweight_forget%f_reg%e_treg%e.csv' %(res1.forget, \
            res1.regularization, res1.t_regularization), valid_times_norm_array, delimiter = ',')
        pbar.update(1)

In [ ]:
plt.plot(np.mean(valid_times_norm_array*0.05,axis = 0))
plt.ylim(0,5)
plt.show()

In [ ]:
x = np.array([[1,2],[3,4]])
np.mean(x,axis = 1)